安装cuda

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
!sudo apt-key add /var/cuda-repo-10-2-local-10.2.89-440.33.01/7fa2af80.pub
!sudo apt-get update
!sudo apt-get -y install cuda

--2020-07-16 11:36:13--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2020-07-16 11:36:13 (3.86 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2020-07-16 11:36:17--  http://developer.download.nvidia.com/compute/cuda/10.2/Prod/local_installers/cuda-repo-ubuntu1804-10-2-local-10.2.89-440.33.01_1.0-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:80... connected.
HTTP request sent, awaiting response...

安装mxnet和gluonnlp

In [3]:
!pip install --upgrade mxnet-cu100==1.6.0 -f https://dist.mxnet.io/python
!pip install --upgrade gluonnlp

Looking in links: https://dist.mxnet.io/python
     |████████████████████████████████| 695.0MB 26kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 256kB 3.3MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470033 sha256=b69fe22f4a17b4f45c01a18900818d27b47566ac9902f4486203ad67cfff7875
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp


In [4]:
import os
os.chdir("/content/drive/My Drive/sentence_embedding")
!ls

aclImdb				    index.rst
aclImdb_v1.tar.gz		    labeledTrainData.tsv
attention-nlp.png		    log_MRPC.txt
base_0.params			    log_SST.txt
base_1.params			    log_STS-B.txt
base_2.params			    lr5_eight_base_0.params
base_3.params			    lr5_eight_base_1.params
base_4.params			    lr5_eight_base_2.params
bert				    output_dir
bert-embed.png			    self_attentive_sentence_embedding.ipynb
BERT_finetune_base.ipynb	    testDataset_8.tsv
bert_imdb.ipynb			    testDataset_multi.tsv
BERT_inference_base.ipynb	    testDataset.tsv
bert.ipynb			    test.tsv
bert.png			    trainDataset_8.tsv
bert-sentence-pair.png		    trainDataset_multi.tsv
Bi-LSTM-Rep.png			    tri_base_0.params
dev.tsv				    tri_base_1.params
eight_base_0.params		    tri_base_2.params
eight_base_1.params		    tri_base_3.params
eight_base_2.params		    tri_base_4.params
eight_base_3.params		    try0.tsv
eight_base_4.params		    Untitled
elmo_sentence_representation.ipynb  Untitled0.ipynb


导入必要的库

In [5]:
import warnings
warnings.filterwarnings('ignore')

import io
import random
import numpy as np
import mxnet as mx
import gluonnlp as nlp
from gluonnlp.calibration import BertLayerCollector
# this notebook assumes that all required scripts are already
# downloaded from the corresponding tutorial webpage on http://gluon-nlp.mxnet.io

from bert import data

nlp.utils.check_version('0.8.1')

设置环境

In [6]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)

导入BERT模型

In [7]:
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                        dataset_name='book_corpus_wiki_en_uncased',
                        pretrained=True, ctx=ctx, use_pooler=True,
                        use_decoder=False, use_classifier=False)
print(bert_base)

Vocab file is not found. Downloading.
BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): DotProductSelfAttentionCell(
          (dropout_layer): Dropout(p = 0.1, axes=())
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): PositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (layer_norm): LayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
      )
      (1): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (at

将模型用于分类

In [8]:
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=8, dropout=0.1)
# only need to initialize the classifier layer.
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)

# softmax cross entropy loss for classification
loss_function = mx.gluon.loss.SoftmaxCELoss()
loss_function.hybridize(static_alloc=True)

metric = mx.metric.Accuracy()

导入数据集

In [9]:
tsv_file = io.open('trainDataset_8.tsv', encoding='utf-8')
for i in range(5):
    print(tsv_file.readline())


id	sentiment	review

5814_8	8	With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature

数据预处理

In [10]:
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
sample_splitter = nlp.data.Splitter('\n')
# Fields to select from the file
field_indices = [2,1]
data_train_raw = nlp.data.TSVDataset(filename='trainDataset_8.tsv',
                  sample_splitter=sample_splitter,
                  field_separator=field_separator,
                  num_discard_samples=num_discard_samples,
                  field_indices=field_indices)
sample_id = 100
# Sentence
print(data_train_raw[sample_id][0])
# label
print(data_train_raw[sample_id][1])


There is a uk edition to this show which is rather less extravagant than the US version. The person concerned will get a new kitchen or perhaps bedroom and bathroom and is wonderfully grateful for what they have got. The US version of this show is everything that reality TV shouldn't be. Instead of making a few improvements to a house which the occupants could not afford or do themselves the entire house gets rebuilt. I do not know if this show is trying to show what a lousy welfare system exists in the US or if you beg hard enough you will receive. The rather vulgar product placement that takes place, particularly by Sears, is also uncalled for. Rsther than turning one family in a deprived area into potential millionaires, it would be far better to help the community as a whole where instead of spending the hundreds of thousands of dollars on one home, build something for the whole community ..... perhaps a place where diy and power tools can be borrowed and returned along with buildi

In [11]:
# Use the vocabulary from pre-trained model for tokenization
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)

# The maximum length of an input sequence
max_len = 128

# The labels for the three classes 
all_labels = ["1", "2", "3", "4", "7", "8", "9", "10"]

# whether to transform the data as sentence pairs.
# for single sentence classification, set pair=False
# for regression task, set class_labels=None
# for inference without label available, set has_label=False
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                        class_labels=all_labels,
                        has_label=True,
                        pad=True,
                        pair=pair)
data_train = data_train_raw.transform(transform)

print('vocabulary used for tokenization = \n%s'%vocabulary)
print('%s token id = %s'%(vocabulary.padding_token, vocabulary[vocabulary.padding_token]))
print('%s token id = %s'%(vocabulary.cls_token, vocabulary[vocabulary.cls_token]))
print('%s token id = %s'%(vocabulary.sep_token, vocabulary[vocabulary.sep_token]))
print('token ids = \n%s'%data_train[sample_id][0])
print('segment ids = \n%s'%data_train[sample_id][1])
print('valid length = \n%s'%data_train[sample_id][2])
print('label = \n%s'%data_train[sample_id][3])

vocabulary used for tokenization = 
Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
[PAD] token id = 1
[CLS] token id = 2
[SEP] token id = 3
token ids = 
[    2  2045  2003  1037  2866  3179  2000  2023  2265  2029  2003  2738
  2625 27856  2084  1996  2149  2544  1012  1996  2711  4986  2097  2131
  1037  2047  3829  2030  3383  5010  1998  5723  1998  2003  6919  2135
  8794  2005  2054  2027  2031  2288  1012  1996  2149  2544  1997  2023
  2265  2003  2673  2008  4507  2694  5807  1005  1056  2022  1012  2612
  1997  2437  1037  2261  8377  2000  1037  2160  2029  1996 18837  2071
  2025  8984  2030  2079  3209  1996  2972  2160  4152  7183  1012  1045
  2079  2025  2113  2065  2023  2265  2003  2667  2000  2265  2054  1037
 10223  6508  7574  2291  6526  1999  1996  2149  2030  2065  2017 11693
  2524  2438  2017  2097  4374  1012  1996  2738 29364  4031 11073  2008
  3138  2173  1010  3391  2011 18493  1010     3]
segment ids = 
[0 0 0 0 0 0 0 0 0

In [12]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 10322582434020903179, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4324898605934026055
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 15717706960708396224
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9836100352
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 18292334621071240024
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

微调

In [ ]:
# The hyperparameters
batch_size = 32
lr = 5e-5

# The FixedBucketSampler and the DataLoader for making the mini-batches
train_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_train],
                                            batch_size=batch_size,
                                            shuffle=True)
bert_dataloader = mx.gluon.data.DataLoader(data_train, batch_sampler=train_sampler)

trainer = mx.gluon.Trainer(bert_classifier.collect_params(), 'adam',
                           {'learning_rate': lr, 'epsilon': 1e-9})

# Collect all differentiable parameters
# `grad_req == 'null'` indicates no gradients are calculated (e.g. constant parameters)
# The gradients for these params are clipped later
params = [p for p in bert_classifier.collect_params().values() if p.grad_req != 'null']
grad_clip = 1

# Training the model with only three epochs
log_interval = 100
num_epochs = 5
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(bert_dataloader):
        with mx.autograd.record():

            # Load the data to the GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # 前馈运算 -> forward
            out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
            # 做均值运算
            ls = loss_function(out, label).mean()

        # And backwards computation
        ls.backward()

        # 梯度截断
        trainer.allreduce_grads()# 梯度累加
        nlp.utils.clip_grad_global_norm(params, 1)# 门限为1
        trainer.update(1) # 因为ls已经mean()了,所以这里的batch_size=1

        step_loss += ls.asscalar()
        metric.update([label], [out])

        # Printing vital information
        if (batch_id + 1) % (log_interval) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.7f}, acc={:.3f}'
                         .format(epoch_id, batch_id + 1, len(bert_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0
    #导出模型
    filename="./lr5_eight_base_"+str(epoch_id)+".params"
    nlp.utils.save_parameters(bert_classifier,filename)

[Epoch 0 Batch 100/1245] loss=1.4154, lr=0.0000500, acc=0.443
[Epoch 0 Batch 200/1245] loss=1.3708, lr=0.0000500, acc=0.454
[Epoch 0 Batch 300/1245] loss=1.3712, lr=0.0000500, acc=0.459
[Epoch 0 Batch 400/1245] loss=1.4057, lr=0.0000500, acc=0.456
[Epoch 0 Batch 500/1245] loss=1.4062, lr=0.0000500, acc=0.451
[Epoch 0 Batch 600/1245] loss=1.3942, lr=0.0000500, acc=0.450
[Epoch 0 Batch 700/1245] loss=1.4142, lr=0.0000500, acc=0.448
[Epoch 0 Batch 800/1245] loss=1.3969, lr=0.0000500, acc=0.447
[Epoch 0 Batch 900/1245] loss=1.3736, lr=0.0000500, acc=0.448
[Epoch 0 Batch 1000/1245] loss=1.3957, lr=0.0000500, acc=0.449
[Epoch 0 Batch 1100/1245] loss=1.3856, lr=0.0000500, acc=0.450
[Epoch 0 Batch 1200/1245] loss=1.3811, lr=0.0000500, acc=0.450
[Epoch 1 Batch 100/1245] loss=1.1634, lr=0.0000500, acc=0.527
[Epoch 1 Batch 200/1245] loss=1.1810, lr=0.0000500, acc=0.525
[Epoch 1 Batch 300/1245] loss=1.2054, lr=0.0000500, acc=0.520
[Epoch 1 Batch 400/1245] loss=1.1856, lr=0.0000500, acc=0.522
[Epoc

测试

测试集

In [ ]:
#加载验证集
# Skip the first line, which is the schema
num_discard_samples = 1
# Split fields by tabs
field_separator = nlp.data.Splitter('\t')
sample_splitter = nlp.data.Splitter('\n')
# Fields to select from the file
field_indices=[2,1]
data_test_raw = nlp.data.TSVDataset(filename='testDataset_8.tsv',
                  sample_splitter=sample_splitter,
                  field_separator=field_separator,
                  num_discard_samples=num_discard_samples,
                  field_indices=field_indices)
data_test = data_test_raw.transform(transform)

In [ ]:
#在验证集上的精度
num_epochs = 5
batch_size = 8
test_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_test],
                      batch_size=batch_size,
                      shuffle=True)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_sampler=test_sampler)

validation = nlp.model.BERTClassifier(bert_base, num_classes=8, dropout=0.1)
validation.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
validation.hybridize(static_alloc=True)

for epoch_id in range(num_epochs):
  filename="./lr5_eight_base_"+str(epoch_id)+".params"
  nlp.utils.load_parameters(validation,filename,ctx=ctx)
  metric.reset()
  for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(test_dataloader):
    with mx.autograd.record():
      # Load the data to the GPU
      token_ids = token_ids.as_in_context(ctx)
      valid_length = valid_length.as_in_context(ctx)
      segment_ids = segment_ids.as_in_context(ctx)
      label = label.as_in_context(ctx)

      # Forward computation
      out = validation(token_ids, segment_ids, valid_length.astype('float32'))
    metric.update([label], [out])
  print('Epoch '+str(epoch_id)+' validation acc:',metric.get()[1])


Epoch 0 validation acc: 0.42848837209302326
Epoch 1 validation acc: 0.4251937984496124
Epoch 2 validation acc: 0.4145348837209302
Epoch 3 validation acc: 0.40930232558139534
Epoch 4 validation acc: 0.43333333333333335


In [ ]:
np.random.seed(100)
random.seed(100)
mx.random.seed(10000)
# change `ctx` to `mx.cpu()` if no GPU is available.
ctx = mx.gpu(0)
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                        dataset_name='book_corpus_wiki_en_uncased',
                        pretrained=True, ctx=ctx, use_pooler=True,
                        use_decoder=False, use_classifier=False)
bert_classifier = nlp.model.BERTClassifier(bert_base, num_classes=8, dropout=0.1)
bert_classifier.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
bert_classifier.hybridize(static_alloc=True)
filename="./lr5_eight_base_4.params"
nlp.utils.load_parameters(bert_classifier,filename,ctx=ctx)

In [ ]:
import os
os.chdir("/content/drive/My Drive/sentence_embedding")
!ls

aclImdb				    labeledTrainData.tsv
aclImdb_v1.tar.gz		    log_MRPC.txt
attention-nlp.png		    log_SST.txt
base_0.params			    log_STS-B.txt
base_1.params			    lr5_eight_base_0.params
base_2.params			    lr5_eight_base_1.params
base_3.params			    lr5_eight_base_2.params
base_4.params			    lr5_eight_base_3.params
bert				    lr5_eight_base_4.params
bert-embed.png			    output_dir
BERT_finetune_base.ipynb	    self_attentive_sentence_embedding.ipynb
bert_imdb.ipynb			    testDataset_8.tsv
BERT_inference_base.ipynb	    testDataset_multi.tsv
bert.ipynb			    testDataset.tsv
bert.png			    test.tsv
bert-sentence-pair.png		    trainDataset_8.tsv
Bi-LSTM-Rep.png			    trainDataset_multi.tsv
dev.tsv				    tri_base_0.params
eight_base_0.params		    tri_base_1.params
eight_base_1.params		    tri_base_2.params
eight_base_2.params		    tri_base_3.params
eight_base_3.params		    tri_base_4.params
eight_base_4.params		    try0.tsv
elmo_sentence_representation.ipynb  Untitled
index.rst			    Untit

In [ ]:
field_separator = nlp.data.Splitter('\t')
# test_sentence = input('please enter the sentence')
# with open('test.csv', 'w', newline='') as csv_file:
#     writer = csv.writer(csv_file, dialect='excel')
#     writer.writerow('test_sentence')
data_inference_row = nlp.data.TSVDataset(filename='test.tsv',
                field_separator=field_separator)
print(data_inference_row[1])
bert_tokenizer = nlp.data.BERTTokenizer(vocabulary, lower=True)
max_len = 128
all_labels = ["1", "2", "3", "4", "7", "8", "9", "10"]
pair = False
transform = data.transform.BERTDatasetTransform(bert_tokenizer, max_len,
                        class_labels=all_labels,
                        has_label=True,
                        pad=True,
                        pair=pair)
data_inference=data_inference_row.transform(transform)
batch_size = 1
data_sampler = nlp.data.FixedBucketSampler(lengths=[int(item[2]) for item in data_inference],
                      batch_size=batch_size,
                      shuffle=True)

['"""How to turn a chewy new flower into a new flower is that it must not be a science fiction suspense. A happy day is a good attempt, as is Palm Springs, but obviously the latter has something more. At least in my opinion, a relationship is an appointment to go to death, without the determination to die, it is better to be trapped in reincarnation."""', '8']


In [ ]:
import math
data_loader = mx.gluon.data.DataLoader(data_inference, batch_sampler=data_sampler)
for batch_id, (token_ids, segment_ids, valid_length, label) in enumerate(data_loader):
  token_ids = token_ids.as_in_context(ctx)
  valid_length = valid_length.as_in_context(ctx)
  segment_ids = segment_ids.as_in_context(ctx)
  label = label.as_in_context(ctx)
  out = bert_classifier(token_ids, segment_ids, valid_length.astype('float32'))
  print(data_inference_row[batch_id][0])
  print('label: ',data_inference_row[batch_id][1])
  print(out)
  delta = 0
  for i in range(8):
    delta += (i-5.5)*out[0][i]
  predict = 5.5 + delta/8
  #print(np.floor(predict))
  print(predict)

  # if (out.argmax(axis=1))[0]==1:
  #   print('inference: 1')
  # else:
  #   print('inference: 0')

"""For the Mother of "The Search for Mom". Compared with other films of the same type, the biggest feature of this film is the unique laziness of the millennial generation. Unfortunately, there are few climaxes in the story, and there are few golden sentences. The length of 90 minutes also has a long sense of 120 minutes."""
label:  7

[[ 2.9200022  -1.0006231  -0.93910044 -1.7661268  -1.4485964  -0.8727535
   0.48444784  3.5936024 ]]
<NDArray 1x8 @gpu(0)>

[6.0483575]
<NDArray 1 @gpu(0)>
"""How to turn a chewy new flower into a new flower is that it must not be a science fiction suspense. A happy day is a good attempt, as is Palm Springs, but obviously the latter has something more. At least in my opinion, a relationship is an appointment to go to death, without the determination to die, it is better to be trapped in reincarnation."""
label:  8

[[-3.3816025 -4.5945015 -3.300514  -1.533656   2.859555   4.007439
   3.237679   3.200744 ]]
<NDArray 1x8 @gpu(0)>

[12.348364]
<NDArray 1 @g